In [1]:
from vdi.settings import settings

CONTROLLER_IP = settings['controller_ip']

from dataclasses import dataclass
from g_tasks import Task, g

from vdi.tasks import Token, disk
from vdi.tasks.ws import WsConnection
from vdi.tasks.client import HttpClient

import urllib, json

class Node(Task):

    async def run(self):
        datapool = await disk.DefaultDatapool()
        nodes = datapool['nodes_connected']
        if len(nodes) == 1:
            [node] = nodes
            if node['connection_status'] == 'SUCCESS':
                return node['id']


@dataclass()
class CreateDomain(Task):

    vm_name: str

    url = f'http://{CONTROLLER_IP}/api/domains/'

    async def params(self):
        node_id = await Node()
        return {
            'cpu_count': 1,
            'cpu_priority': "10",
            'memory_count': 1024,
            'node': node_id,
            'os_type': "Other",
            'sound': {'model': "ich6", 'codec': "micro"},
            'verbose_name': self.vm_name,
            'video': {'type': "cirrus", 'vram': "16384", 'heads': "1"},
        }

    def is_done(self, msg):
        obj = msg['object']
        if not obj['status'] == 'SUCCESS':
            return
        for id, e in obj['entities'].items():
            if e == 'domain':
                return id == self.domain['id']

    async def run(self):
        token = await Token()
        headers = {
            'Authorization': f'jwt {token}'
        }
        ws = await WsConnection()
#         await ws.send('add /tasks/')
        http_client = HttpClient()
        params = await self.params()
        body = urllib.parse.urlencode(params)
        response = await http_client.fetch(self.url, method='POST', headers=headers, body=body)
        self.domain = json.loads(response.body)
        print(f"fetched: {self.domain['id']}")
        await ws.wait_message(self.is_done)
        return self.domain


In [2]:
g.init()
await CreateDomain(vm_name='kviyam')

fetched: e13b6107-1f0a-436d-93f7-abb1328168a9
> /home/pwtail/projects/vdi2/vdi/tasks/ws.py(39)wait_message()
     38             breakpoint()
---> 39             if hasattr(task, 'i_am_timeout'):
     40                 ws_task.cancel()



ipdb>  p task


<Task finished coro=<WsConnection.make_sleep_task.<locals>.co() done, defined at /home/pwtail/projects/vdi2/vdi/tasks/ws.py:54> result=None>


ipdb>  p task.i_am_timeout


True


ipdb>  c


WsTimeout: 

In [1]:
from vdi.asyncio_utils import Wait

import asyncio
t1 = asyncio.create_task(asyncio.sleep(1))
t2 = asyncio.create_task(asyncio.sleep(2))

vals = []

async for r, t in Wait(t1, t2).items():
    vals.append((r, t))

In [2]:
vals

[(None,
  <Task finished coro=<sleep() done, defined at /usr/lib64/python3.7/asyncio/tasks.py:555> result=None>),
 (None,
  <Task finished coro=<sleep() done, defined at /usr/lib64/python3.7/asyncio/tasks.py:555> result=None>)]

In [3]:
vals[1][1]

<Task finished coro=<sleep() done, defined at /usr/lib64/python3.7/asyncio/tasks.py:555> result=None>

In [16]:
_3.ff = 9


In [11]:
_.cr_code

<code object sleep at 0x7f8792db0d20, file "/usr/lib64/python3.7/asyncio/tasks.py", line 555>